<a href="https://colab.research.google.com/github/kseohyuz/Toi_CNN2/blob/main/TOI_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **<FINAL PROJECT: 실시간 얼굴 탐지를 통한 영어 이름 추천>**

1. Mediapipe으로 얼굴 인식된 이미지를 CNN 모델에 전달
2. CNN 모델의 예측 결과(성별, 연령대)를 기반으로
3. 영어 이름 추천 함수를 호출하여 추천 결과를 제공

### **1. CNN 모델 학습을 위한 데이터 준비**

1.1. 데이터 다운로드

In [ ]:
!git clone https://github.com/imdeepmind/processed-imdb-wiki-dataset.git

Cloning into 'processed-imdb-wiki-dataset'...
remote: Enumerating objects: 137, done.
remote: Total 137 (delta 0), reused 0 (delta 0), pack-reused 137 (from 1)
Receiving objects: 100% (137/137), 47.05 MiB | 20.49 MiB/s, done.
Resolving deltas: 100% (73/73), done.


In [ ]:
!pip install numpy scipy pandas opencv-python

In [ ]:
!wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar
!tar -xvf imdb_crop.tar -C /content

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
imdb_crop/05/nm0192505_rm2424942336_1985-11-30_2007.jpg
imdb_crop/05/nm0192505_rm2540822272_1985-11-30_2007.jpg
imdb_crop/05/nm0192505_rm2691354880_1985-11-30_2007.jpg
imdb_crop/05/nm0192505_rm2738591744_1985-11-30_2007.jpg
imdb_crop/05/nm0192505_rm2756282624_1985-11-30_2009.jpg
imdb_crop/05/nm0192505_rm2761936896_1985-11-30_2007.jpg
imdb_crop/05/nm0192505_rm2769718784_1985-11-30_2010.jpg
imdb_crop/05/nm0192505_rm279885312_1985-11-30_2007.jpg
imdb_crop/05/nm0192505_rm2805700608_1985-11-30_2007.jpg
imdb_crop/05/nm0192505_rm2856032256_1985-11-30_2007.jpg
imdb_crop/05/nm0192505_rm2866577408_1985-11-30_2007.jpg
imdb_crop/05/nm0192505_rm2877203200_1985-11-30_2007.jpg
imdb_crop/05/nm0192505_rm2917773312_1985-11-30_2007.jpg
imdb_crop/05/nm0192505_rm2926235904_1985-11-30_2007.jpg
imdb_crop/05/nm0192505_rm2961024512_1985-11-30_2007.jpg
imdb_crop/05/nm0192505_rm3030760192_1985-11-30_2007.jpg
imdb_crop/05/nm0192505_rm3087642368_1985-11-30_2007.jpg
imdb_crop/05/

In [ ]:
!ls /content/imdb_crop

00  01	02  03	04  05	06


1.2. 데이터 구조 이해 및 메타데이터 읽기

In [ ]:
from scipy.io import loadmat
imdb_meta = loadmat('/content/imdb_crop/imdb.mat')

FileNotFoundError: [Errno 2] No such file or directory: '/content/imdb_crop/imdb.mat'

In [ ]:
import scipy.io
import pandas as pd

# 메타데이터 로드
meta = scipy.io.loadmat('/content/imdb_crop/imdb.mat')

# 필요한 정보 추출
image_paths = meta['imdb'][0][0][2][0]
genders = meta['imdb'][0][0][3][0]
ages = meta['imdb'][0][0][6][0]

# 데이터프레임 생성
data = pd.DataFrame({'image_path': image_paths, 'gender': genders, 'age': ages})

In [ ]:
data

1.3. 데이터 필터링 및 균등 샘플링

In [ ]:
#라벨링 함수 정의
def age_label(age):
    if age < 10:
        return '0s'
    elif age < 20:
        return '10s'
    elif age < 30:
        return '20s'
    elif age < 40:
        return '30s'
    elif age < 50:
        return '40s'
    elif age < 60:
        return '50s'
    else:
        return '60s'

In [ ]:
#라벨링 적용 및 결측치 제거
data['age_label'] = data['age'].apply(age_label)
data = data[(data['age'] > 0) & (data['age'] < 70)]
data = data.dropna()

In [ ]:
from sklearn.utils import resample

def balanced_sample(data, n_samples):
    sampled_data = pd.DataFrame()
    for gender in [0, 1]:  # 0: 여성, 1: 남성
        for age_group in ['0s', '10s', '20s', '30s', '40s', '50s', '60s']:
            subset = data[(data['gender'] == gender) & (data['age_label'] == age_group)]
            if len(subset) > n_samples:
                subset = resample(subset, n_samples=n_samples, random_state=42)
            sampled_data = pd.concat([sampled_data, subset])
    return sampled_data

# 각 성별과 연령대에서 균등하게 500개의 샘플을 추출 (필요에 따라 조정 가능)
balanced_data = balanced_sample(data, n_samples=500)

1.5. 데이터 크롭 (얼굴영역만 크롭)

1.6. 데이터 정규화

1.7. 데이터 분리

1.8. 데이터 저장 및 이미지 증강

### **2. CNN 모델 학습**

2.1. 전이 학습을 위한 CNN 모델 학습

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 랜덤 시드 고정 (재현성을 위해)
np.random.seed(42)
tf.random.set_seed(42)

# 하이퍼파라미터 설정
num_classes = 10  # 클래스 수에 맞게 조정
image_size = (224, 224)
batch_size = 32
epochs = 50
learning_rate = 1e-4

# 데이터 로드 (예시 - 실제 데이터로 대체 필요)
# X_train, y_train, X_val, y_val, X_test, y_test = load_your_data()

# 데이터 증강
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 검증 데이터 분할
)

# ResNet50 기반 모델 구축
def create_transfer_learning_model(num_classes):
    # 기본 ResNet50 모델 로드 (ImageNet 가중치)
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(*image_size, 3)
    )

    # 기본 모델 레이어 동결
    for layer in base_model.layers:
        layer.trainable = False

    # 추가 레이어 구성
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)  # 과적합 방지를 위한 드롭아웃
    predictions = Dense(num_classes, activation='softmax')(x)

    # 최종 모델 생성
    model = Model(inputs=base_model.input, outputs=predictions)

    return model, base_model

# 모델 생성
model, base_model = create_transfer_learning_model(num_classes)

# 모델 컴파일
model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 콜백 설정
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True
)

# 미세 조정 단계 (후반부 레이어 해동)
def fine_tune_model(model, base_model):
    # 일부 베이스 모델 레이어 해동
    for layer in base_model.layers[-50:]:
        layer.trainable = True

    # 낮은 학습률로 컴파일
    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# 모델 훈련
def train_model(model, X_train, y_train, X_val, y_val):
    # 초기 훈련
    history_initial = model.fit(
        datagen.flow(X_train, y_train, batch_size=batch_size),
        epochs=epochs//2,
        validation_data=datagen.flow(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint]
    )

    # 미세 조정
    model = fine_tune_model(model, base_model)

    history_fine_tune = model.fit(
        datagen.flow(X_train, y_train, batch_size=batch_size),
        epochs=epochs//2,
        validation_data=datagen.flow(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint]
    )

    return model, history_initial, history_fine_tune

# 성능 평가
def evaluate_model(model, X_test, y_test):
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print(f'Test Loss: {test_loss}')
    print(f'Test Accuracy: {test_accuracy}')

# 메인 실행 (주의: 실제 데이터로 대체 필요)
# trained_model, history_initial, history_fine_tune = train_model(model, X_train, y_train, X_val, y_val)
# evaluate_model(trained_model, X_test, y_test)

# 모델 저장
# model.save('final_cnn_model.h5')

2.2. 테스트 데이터로 모델 평가

### **3. Face Recognition**

3.1. OpenCV 활용 카메라(웹캠) 테스트

In [ ]:
import cv2
import numpy as np

def test_webcam():
    # 웹캠 캡처 객체 생성 (0은 기본 웹캠, 추가 웹캠이 있다면 1, 2 등으로 변경)
    cap = cv2.VideoCapture(0)

    # 웹캠 해상도 설정 (선택사항)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    # 웹캠 연결 확인
    if not cap.isOpened():
        print("웹캠을 열 수 없습니다.")
        return

    while True:
        # 프레임 읽기
        ret, frame = cap.read()

        # 프레임 읽기 실패 시 루프 종료
        if not ret:
            print("프레임을 읽을 수 없습니다.")
            break

        # 프레임에 텍스트 추가 (선택사항)
        cv2.putText(frame,
                    f'Webcam Test',
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    (0, 255, 0),
                    2)

        # 추가 효과 - 그레이스케일 변환 (좌측에 표시)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # 이미지 좌우 분할 및 비교
        h, w = frame.shape[:2]
        frame[:, :w//2] = gray[:, :w//2, np.newaxis]

        # 프레임 표시
        cv2.imshow('Webcam Test', frame)

        # 키보드 입력 처리
        key = cv2.waitKey(1) & 0xFF

        # 'q' 키를 누르면 종료
        if key == ord('q'):
            break
        # 's' 키를 누르면 스크린샷 저장
        elif key == ord('s'):
            cv2.imwrite('webcam_screenshot.png', frame)
            print("스크린샷 저장됨")

    # 리소스 해제
    cap.release()
    cv2.destroyAllWindows()

def detect_faces():
    # 얼굴 인식 추가 함수
    # 미리 학습된 Haar Cascade 분류기 로드
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 그레이스케일 변환
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # 얼굴 검출
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        # 검출된 얼굴에 사각형 그리기
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, 'Face', (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        cv2.imshow('Face Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def main():
    print("1. 기본 웹캠 테스트")
    print("2. 얼굴 인식 테스트")

    choice = input("선택 (1/2): ")

    if choice == '1':
        test_webcam()
    elif choice == '2':
        detect_faces()
    else:
        print("잘못된 선택입니다.")

if __name__ == "__main__":
    main()

3.2. Mediapipe 실시간 얼굴 검출

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

class FaceDetector:
    def __init__(self,
                 min_detection_confidence=0.5,
                 min_tracking_confidence=0.5):
        """
        MediaPipe 얼굴 검출기 초기화

        :param min_detection_confidence: 최소 검출 신뢰도
        :param min_tracking_confidence: 최소 추적 신뢰도
        """
        # MediaPipe 얼굴 감지 모듈 초기화
        self.mp_face_detection = mp.solutions.face_detection
        self.mp_drawing = mp.solutions.drawing_utils

        # 얼굴 감지기 생성
        self.face_detection = self.mp_face_detection.FaceDetection(
            min_detection_confidence=min_detection_confidence,
            min_tracking_confidence=min_tracking_confidence
        )

    def detect_faces(self, image):
        """
        이미지에서 얼굴 검출

        :param image: OpenCV 이미지
        :return: 얼굴 경계 박스와 랜드마크가 표시된 이미지
        """
        # BGR to RGB 변환
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 얼굴 검출 처리
        results = self.face_detection.process(image_rgb)

        # 결과가 있다면 얼굴에 박스와 랜드마크 그리기
        if results.detections:
            for detection in results.detections:
                # 바운딩 박스 좌표 변환
                ih, iw, _ = image.shape
                bbox = detection.location_data.relative_bounding_box

                # 상대 좌표를 절대 좌표로 변환
                x = int(bbox.xmin * iw)
                y = int(bbox.ymin * ih)
                w = int(bbox.width * iw)
                h = int(bbox.height * ih)

                # 바운딩 박스 그리기
                cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

                # 신뢰도 표시
                confidence = detection.score[0]
                cv2.putText(image,
                            f'Face: {confidence:.2f}',
                            (x, y-10),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.9,
                            (0, 255, 0),
                            2)

                # 얼굴 랜드마크 그리기
                self.mp_drawing.draw_detection(image, detection)

        return image

def real_time_face_detection():
    """
    실시간 얼굴 검출 함수
    """
    # 웹캠 캡처 객체 생성
    cap = cv2.VideoCapture(0)

    # 얼굴 검출기 생성
    face_detector = FaceDetector()

    while True:
        # 프레임 읽기
        ret, frame = cap.read()

        if not ret:
            break

        # 프레임 미러링 (좌우 반전)
        frame = cv2.flip(frame, 1)

        # 얼굴 검출
        result_frame = face_detector.detect_faces(frame)

        # 추가 정보 표시
        cv2.putText(result_frame,
                    'MediaPipe Face Detection',
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    (0, 255, 0),
                    2)

        # 프레임 표시
        cv2.imshow('Real-time Face Detection', result_frame)

        # 키보드 입력 처리
        key = cv2.waitKey(1) & 0xFF

        # 'q' 키를 누르면 종료
        if key == ord('q'):
            break
        # 's' 키를 누르면 스크린샷 저장
        elif key == ord('s'):
            cv2.imwrite('face_detection_screenshot.png', result_frame)
            print("스크린샷 저장됨!")

    # 리소스 해제
    cap.release()
    cv2.destroyAllWindows()

def face_landmarks_detection():
    """
    얼굴 랜드마크 검출 함수
    """
    # MediaPipe 얼굴 메시 모듈 초기화
    mp_face_mesh = mp.solutions.face_mesh
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles

    # 웹캠 캡처
    cap = cv2.VideoCapture(0)

    # 얼굴 메시 설정
    with mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as face_mesh:

        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("카메라를 찾을 수 없습니다.")
                continue

            # 이미지 미러링
            image = cv2.flip(image, 1)

            # BGR to RGB 변환
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # 이미지 처리
            results = face_mesh.process(image_rgb)

            # 랜드마크 그리기
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    # 얼굴 윤곽 랜드마크 그리기
                    mp_drawing.draw_landmarks(
                        image=image,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_CONTOURS,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles
                        .get_default_face_mesh_contours_style())

                    # 홍채 랜드마크 그리기
                    mp_drawing.draw_landmarks(
                        image=image,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_IRISES,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles
                        .get_default_face_mesh_iris_connections_style())

            cv2.imshow('Face Mesh', image)

            # 'q' 키를 누르면 종료
            if cv2.waitKey(5) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

def main():
    print("MediaPipe 얼굴 검출 옵션:")
    print("1. 실시간 얼굴 검출")
    print("2. 얼굴 랜드마크 검출")

    choice = input("선택 (1/2): ")

    if choice == '1':
        real_time_face_detection()
    elif choice == '2':
        face_landmarks_detection()
    else:
        print("잘못된 선택입니다.")

if __name__ == "__main__":
    main()

###**4. 영어 이름 추천**

4.1. 영어 이름 추천을 위한 딕셔너리 구축

In [ ]:
import pandas as pd
from sklearn.utils import resample

def balanced_sample(data, n_samples):
    sampled_data = pd.DataFrame()
    for gender in [0, 1]:  # 0: 여성, 1: 남성
        for age_group in ['0s', '10s', '20s', '30s', '40s', '50s', '60s']:
            subset = data[(data['gender'] == gender) & (data['age_label'] == age_group)]
            if len(subset) > n_samples:
                subset = resample(subset, n_samples=n_samples, random_state=42)
            sampled_data = pd.concat([sampled_data, subset])
    return sampled_data

def recommend_name(data, gender, age_group, style):
    """
    이름 추천 함수
    :param data: 이름 데이터 딕셔너리
    :param gender: 성별 (0: 여성, 1: 남성)
    :param age_group: 나이 그룹 ('0s', '10s', '20s', '30s', '40s', '50s', '60s')
    :param style: 이름 스타일 ('modern', 'traditional')
    :return: 추천 이름 리스트
    """
    # gender와 age_group을 문자열로 변환
    gender_str = "female" if gender == 0 else "male"

    # 연령대 매핑
    age_mapping = {
        '0s': 'child',
        '10s': 'teen',
        '20s': 'adult',
        '30s': 'adult',
        '40s': 'adult',
        '50s': 'senior',
        '60s': 'senior'
    }

    mapped_age_group = age_mapping.get(age_group, 'adult')

    return data[gender_str].get(mapped_age_group, {}).get(style, [])

# 이름 데이터name_data = {
    "male": {
        "child": {
            "modern": [
                "Liam", "Noah", "Jackson", "Ethan", "Mason", "Logan", "Elijah", "Aiden", "Jameson", "Wyatt",
                "Oliver", "Lucas", "Leo", "Jack", "Benjamin", "Alexander", "Sebastian", "Owen", "Michael", "Kai"
            ],
            "traditional": [
                "William", "James", "Henry", "Charles", "John", "George", "Samuel", "Benjamin", "Edward", "Daniel",
                "Thomas", "Robert", "Joseph", "David", "Matthew", "Andrew", "Christopher", "Jonathan", "Nicholas", "Peter"
            ]
        },
        "teen": {
            "modern": [
                "Aiden", "Lucas", "Grayson", "Caleb", "Connor", "Hunter", "Julian", "Owen", "Eli", "Asher",
                "Jayden", "Ryan", "Cameron", "Nathan", "Zach", "Isaac", "Evan", "Tyler", "Gabriel", "Kai"
            ],
            "traditional": [
                "Michael", "Joseph", "David", "Andrew", "Patrick", "Thomas", "Christopher", "Matthew", "Anthony", "Nicholas",
                "Daniel", "Robert", "John", "James", "William", "Steven", "Kevin", "Mark", "Brian", "Richard"
            ]
        },
        "adult": {
            "modern": [
                "Ethan", "Mason", "Oliver", "Carter", "Nathan", "Ryan", "Dylan", "Landon", "Levi", "Isaac",
                "Caleb", "Wyatt", "Kai", "Axel", "Hunter", "Jason", "Zander", "Finn", "Rowan", "Sebastian"
            ],
            "traditional": [
                "Robert", "Charles", "Mark", "Richard", "Steven", "Paul", "Kenneth", "Roger", "Phillip", "Arthur",
                "Donald", "Timothy", "Larry", "Scott", "Jeffrey", "Douglas", "Gary", "Frank", "Dennis", "Gerald"
            ]
        },
        "senior": {
            "modern": [
                "Oliver", "Henry", "Theodore", "Sebastian", "Ezekiel", "Emmett", "Everett", "Declan", "Roman", "Silas",
                "Graham", "Walter", "Frank", "Clark", "Albert", "Harold", "Leonard", "Earl", "Russell", "Chester"
            ],
            "traditional": [
                "Edward", "George", "Albert", "Frank", "Harold", "Walter", "Louis", "Howard", "Eugene", "Clarence",
                "Raymond", "Donald", "Arthur", "Earl", "Herbert", "Stanley", "Warren", "Milton", "Lloyd", "Norman"
            ]
        }
    },
    "female": {
        "child": {
            "modern": [
                "Sophia", "Mia", "Aria", "Isabella", "Ava", "Luna", "Ella", "Harper", "Scarlett", "Layla",
                "Zoe", "Emma", "Lily", "Nova", "Willow", "Emilia", "Elena", "Aurora", "Stella", "Maya"
            ],
            "traditional": [
                "Elizabeth", "Mary", "Margaret", "Catherine", "Charlotte", "Eleanor", "Anna", "Caroline", "Jane", "Victoria",
                "Grace", "Helen", "Ruth", "Dorothy", "Frances", "Alice", "Evelyn", "Florence", "Edith", "Louise"
            ]
        },
        "teen": {
            "modern": [
                "Olivia", "Emma", "Amelia", "Chloe", "Emily", "Lily", "Hannah", "Zoe", "Addison", "Grace",
                "Aria", "Scarlett", "Madison", "Abigail", "Aubrey", "Riley", "Layla", "Camila", "Hazel", "Mila"
            ],
            "traditional": [
                "Grace", "Margaret", "Helen", "Diana", "Evelyn", "Clara", "Theresa", "Susan", "Martha", "Irene",
                "Jean", "Ann", "Patricia", "Judith", "Rose", "Marie", "Joyce", "Carol", "Nancy", "Sharon"
            ]
        },
        "adult": {
            "modern": [
                "Isabella", "Chloe", "Sophia", "Madison", "Abigail", "Ella", "Avery", "Maya", "Brooklyn", "Hailey",
                "Aria", "Luna", "Nova", "Willow", "Ember", "Scarlett", "Zara", "Eden", "Quinn", "Autumn"
            ],
            "traditional": [
                "Anne", "Katherine", "Rebecca", "Laura", "Victoria", "Julia", "Sarah", "Christine", "Joanna", "Linda",
                "Susan", "Carol", "Margaret", "Nancy, "Sharon", "Karen", "Donna", "Michelle", "Sandra", "Barbara"
            ]
        },
        "senior": {
            "modern": [
                "Charlotte", "Eleanor", "Hazel", "Audrey", "Nora", "Maeve", "Vivian", "Ivy", "Penelope", "Alice",
                "Rose", "Ruth", "Pearl", "Winifred", "Mabel", "Josephine", "Harriet", "Beatrice", "Ethel", "Clara"
            ],
            "traditional": [
                "Beatrice", "Dorothy", "Florence", "Mabel", "Edith", "Rose", "Lillian", "Agnes", "Clara", "Irene",
                "Bertha", "Mildred", "Gladys, "Doris", "Helen", "Esther, "Ruth", "Thelma", "Martha", "Eleanor"
            ]
        }
    }
}

# 사용 예시
# 주의: 실제 사용 시에는 'data' 변수가 정의되어 있어야 합니다.
# balanced_data = balanced_sample(data, n_samples=500)

# 실제 사용 예시
balanced_data = balanced_sample(data, n_samples=500)

# 이름 추천 예시
print(recommend_name(name_data, 1, '0s', "modern"))  # 남성, 0세 연령대, modern 스타일
print(recommend_name(name_data, 0, '60s', "traditional"))  # 여성, 60세 연령대, traditional 스타일


4.2. Mediapipe로 검출한 얼굴을 CNN 모델에 전달

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

class FaceCNNClassifier:
    def __init__(self,
                 model_path,
                 classes=['class1', 'class2', 'class3'],  # 클래스 이름 수정 필요
                 min_detection_confidence=0.5,
                 min_tracking_confidence=0.5):
        """
        MediaPipe 얼굴 검출 및 CNN 분류기 초기화

        :param model_path: 사전 학습된 CNN 모델 경로
        :param classes: 분류할 클래스 목록
        :param min_detection_confidence: 최소 검출 신뢰도
        :param min_tracking_confidence: 최소 추적 신뢰도
        """
        # MediaPipe 얼굴 감지 모듈 초기화
        self.mp_face_detection = mp.solutions.face_detection
        self.mp_drawing = mp.solutions.drawing_utils

        # 얼굴 감지기 생성
        self.face_detection = self.mp_face_detection.FaceDetection(
            min_detection_confidence=min_detection_confidence,
            min_tracking_confidence=min_tracking_confidence
        )

        # CNN 모델 로드
        self.model = load_model(model_path)
        self.classes = classes

    def preprocess_face(self, face_image, target_size=(224, 224)):
        """
        얼굴 이미지 전처리

        :param face_image: 얼굴 이미지
        :param target_size: 모델 입력을 위한 이미지 크기
        :return: 전처리된 이미지 배열
        """
        # 이미지 크기 조정
        face_resized = cv2.resize(face_image, target_size)

        # 이미지를 배열로 변환
        face_array = img_to_array(face_resized)

        # 배치 차원 추가 및 정규화
        face_array = np.expand_dims(face_array, axis=0)
        face_array /= 255.0

        return face_array

    def detect_and_classify_faces(self, image):
        """
        이미지에서 얼굴 검출 및 분류

        :param image: OpenCV 이미지
        :return: 분류 결과가 표시된 이미지
        """
        # BGR to RGB 변환
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 얼굴 검출 처리
        results = self.face_detection.process(image_rgb)

        # 결과가 있다면 얼굴에 박스와 분류 결과 그리기
        if results.detections:
            for detection in results.detections:
                # 바운딩 박스 좌표 변환
                ih, iw, _ = image.shape
                bbox = detection.location_data.relative_bounding_box

                # 상대 좌표를 절대 좌표로 변환
                x = int(bbox.xmin * iw)
                y = int(bbox.ymin * ih)
                w = int(bbox.width * iw)
                h = int(bbox.height * ih)

                # 얼굴 이미지 추출
                face_image = image[y:y+h, x:x+w]

                try:
                    # 얼굴 이미지 전처리 및 분류
                    processed_face = self.preprocess_face(face_image)
                    predictions = self.model.predict(processed_face)

                    # 가장 높은 확률의 클래스 선택
                    class_index = np.argmax(predictions[0])
                    predicted_class = self.classes[class_index]
                    confidence = predictions[0][class_index]

                    # 바운딩 박스 및 분류 결과 그리기
                    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    label = f'{predicted_class}: {confidence:.2f}'
                    cv2.putText(image,
                                label,
                                (x, y-10),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                0.9,
                                (0, 255, 0),
                                2)

                except Exception as e:
                    print(f"얼굴 분류 중 오류 발생: {e}")

        return image

def real_time_face_classification(model_path):
    """
    실시간 얼굴 분류 함수

    :param model_path: 사전 학습된 CNN 모델 경로
    """
    # 웹캠 캡처 객체 생성
    cap = cv2.VideoCapture(0)

    # 얼굴 분류기 생성
    face_classifier = FaceCNNClassifier(
        model_path=model_path,
        classes=['angry', 'happy', 'sad', 'neutral']  # 실제 클래스에 맞게 수정
    )

    while True:
        # 프레임 읽기
        ret, frame = cap.read()

        if not ret:
            break

        # 프레임 미러링 (좌우 반전)
        frame = cv2.flip(frame, 1)

        # 얼굴 검출 및 분류
        result_frame = face_classifier.detect_and_classify_faces(frame)

        # 추가 정보 표시
        cv2.putText(result_frame,
                    'Face Classification',
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    (0, 255, 0),
                    2)

        # 프레임 표시
        cv2.imshow('Real-time Face Classification', result_frame)

        # 키보드 입력 처리
        key = cv2.waitKey(1) & 0xFF

        # 'q' 키를 누르면 종료
        if key == ord('q'):
            break
        # 's' 키를 누르면 스크린샷 저장
        elif key == ord('s'):
            cv2.imwrite('face_classification_screenshot.png', result_frame)
            print("스크린샷 저장됨!")

    # 리소스 해제
    cap.release()
    cv2.destroyAllWindows()

def main():
    # 사전 학습된 모델 경로 (실제 모델 경로로 변경 필요)
    MODEL_PATH = 'path/to/your/trained_cnn_model.h5'

    try:
        real_time_face_classification(MODEL_PATH)
    except Exception as e:
        print(f"오류 발생: {e}")

if __name__ == "__main__":
    main()